# 8章 ディープラーニング
## 8.1 ネットワークをより深く

### 8.1.1 よりディープなネットワークへ
* VGGと呼ばれるネットワークを参考に，よりディープなネットワークを作る
  * 層がより深くなった
  * フィルターは3$\times$3の小さなフィルターを使う
  * 層が深くなるにつれ，チャンネル数を大きくする
    * 畳み込み層のチャンネル数が16,16,32,32,64,64と増えていく
  * 途中でプーリング層を挿入し，中間データの空間サイズを徐々に小さくする
  * 全結合層ではDropoutレイヤを使用する
  * 活性化関数は ReLU を使用する
  * 重みの初期値としてHeの初期値を使用する
  * 重みパラメータの更新はAdamの手法を使う

In [5]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
sys.path.append(os.pardir + "/deep_learning_from_scratch")

import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *

import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

In [6]:
# ディープなネットワークの実装コード(ch08/deep_convnet.pyより)

class DeepConvNet:
    """認識率99%以上の高精度なConvNet

    ネットワーク構成は下記の通り
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 重みの初期化===========
        # 各層のニューロンひとつあたりが、前層のニューロンといくつのつながりがあるか（TODO:自動で計算する）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLUを使う場合に推奨される初期値
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # レイヤの生成===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

### 8.1.2 さらに認識精度を高めるには
* Webサイト「[What is the class of this image?](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html)」には様々なデータセットを対象に論文で発表されてきた数々の手法の精度がランキングで掲載されている
  * ランキングの上位はCNNをベースとしたものが多い
  * 文字認識などの比較的単純な問題の場合，それほど層が深くないCNNでも成績が高い．層を深くすることの精度向上へ貢献度合いは，複雑な問題の方が高い
  * 精度を高めるための技術についてヒントを得られる．たとえばアンサンブル学習とか，学習係数の減衰とか，データ拡張(Data Augmentation)とか

#### データ拡張（Data Augmentation）
* 入力画像(訓練用画像)をアルゴリズムで人工的に拡張する
  * 入力画像に対して回転や移動などの微小変化を与えて画像枚数を増やす．データセットが限られる場合には有効な手法
* 画像の拡張方法も色々ある
  * crop処理（一部を切り出す），flip処理（左右反転させる），輝度を変化させる，拡大・縮小する，など

### 8.1.3 層を深くすることのモチベーション
* 深くすることの重要性は，理論的にあまりわかっていない
* ILSVRCなどのコンペの結果から重要性を汲み取ることができる
  * 上位を占める手法の多くがディープラーニングで，層をより深くする方向に向かっている
* 層を深くする利点
  * 層を深くしなかった場合に比べてより少ないパラメータで同等以上の表現力を達成できる
    * 出力データの特定要素の値に影響を与える入力データの要素の範囲は，5x5のフィルターの畳み込み1層と3x3の畳み込み2層とで同等．後者の方がパラメータ数が少ない（前者は25個，後者は18個）
  * 受容野(receptive field, ニューロンに変化を生じさせる局所的な空間領域)を広くカバーできる
  * 層を深くするとReLUのような非線形の活性化関数が畳み込みの間に挟まれることにより表現力が向上する
  * 層を深くすることで学習データを少なくできる高速に学習できる
* ただし，層を深くできるのは，正しく学習できる技術や計算機能力の向上が前提として存在するからということに注意

## 8.2 ディープラーニングの小歴史
* 最近のディープラーニングのトレンドについて
* ディープラーニングが注目を集めるきっかけ: [ILSVRC(ImageNet Large Scale Visual Recognition Challenge)](http://www.image-net.org/challenges/LSVRC/)のAlexNetが2012年にディープラーニング手法で圧倒的成績で優勝したこと

### 8.2.1 [ImageNet](http://www.image-net.org)
* 画像データセット．
* 様々な種類の，100万枚を超える画像を含む．
* 画像にはラベル(クラス名)が紐づけられている
* ILSVRCというコンペではこのデータセットを使用している
  * 様々なテスト項目があり，その一つに「クラス分類(classification)」がある
  * 最近の結果
    * 2012年を境にディープラーニングが常にトップ
    * 2012年以降，着実に精度が改善されている
    * 2015年のResNetは誤認識率3.5%を達成し，人間の認識能力を上回ったとまで言われた

### 8.2.2 VGG
* CNNの一つ
* 畳み込み層とプーリング層から構成される

### 8.2.3 GoogLeNet
### 8.2.4 ResNet
## 8.3 ディープラーニングの高速化
### 8.3.1 取り組むべき問題
### 8.3.2 GPUによる高速化
### 8.3.3 分散学習
### 8.3.4 演算精度のビット削減
## 8.4 ディープラーニングの実用例
### 8.4.1 物体検出
### 8.4.2 セグメンテーション
### 8.4.3 画像キャプション生成
## 8.5 ディープラーニングの未来
### 8.5.1 画像スタイル変換
### 8.5.2 画像生成
### 8.5.3 自動運転
### 8.5.4 Deep Q-Network(強化学習)

## 8.6 まとめ
* この章では，ディープなCNNを実装し，手書き数字認識で99％を超える認識結果を得た
* ネットワークをディープにすることのモチベーション
  * 多くの問題ではネットワークを深くすることで性能向上が期待できる
* 最近のディープラーニングはディープな方向へと向かっている
  * ILSVRCという画像認識コンペの最近の動向はディープラーニングによる手法が上位を独占し，使われるネットワークもディープになっている
  * 有名なネットワークとしてVGG, GoogLeNet, ResNetがある
* ディープラーニングのトレンド，実用例，高速化に向けた研究や未来の研究例の紹介
  * GPUや分散学習，ビット精度の削減などにより高速化を実現できる
  * ディープラーニングは物体認識以外にも物体検出やセグメンテーションに利用できる
  * ディープラーニングを用いた応用として画像のキャプション生成，画像生成，強化学習などがある．最近では自動運転への利用も期待されている
* 今後
  * ディープラーニングについてまだわかっていないことが多い
  * 新しい研究が次々と発表されている
  * これからも研究が活発に続けられ，新しい技術が現実化されてゆくと思われる

